<a href="https://colab.research.google.com/github/hmin27/2023_DL_Clip/blob/main/CLIP(Finetune)_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIP Fine tuning
- Food image classification
- Baseline of Fine Tuned CLIP model


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [22]:
import os
import clip
import torch
from torch import nn, optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import shutil

%matplotlib inline
BATCH_SIZE = 4
EPOCH = 5
LR = 1e-7

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Prepare the Model and Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model, preprocess = clip.load("ViT-B/16", device=device, jit=False)
model = model.to(torch.float32)

100%|███████████████████████████████████████| 335M/335M [00:11<00:00, 31.2MiB/s]


In [15]:
# Few-shot learning

class FewshotDataset(Dataset):
    def __init__(self, data_folder, preprocess):
        self.data_folder = data_folder
        self.preprocess = preprocess
        self.image_paths = []
        self.text_descriptions = []
        self.labels = []

        # image path list, text list
        class_folders = os.listdir(data_folder)

        for label, class_folder in enumerate(class_folders):
            class_folder_path = os.path.join(data_folder, class_folder)
            image_files = os.listdir(class_folder_path)

            for image_file in image_files:
                image_path = os.path.join(class_folder_path, image_file)
                self.image_paths.append(image_path)

                text_description = f"a photo of {class_folder.replace('_',' ')}"
                self.text_descriptions.append(text_description)

                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        image = self.preprocess(image)
        text = self.text_descriptions[idx]
        label = self.labels[idx]
        return image, text, label


data_folder = '/content/drive/MyDrive/Study/DL_CLIP/Food_fewshot/4_shot'
train_dataset = FewshotDataset(os.path.join(data_folder, 'train'), preprocess)
val_dataset = FewshotDataset(os.path.join(data_folder, 'validation'), preprocess)
test_dataset = FewshotDataset(os.path.join(data_folder, 'test'), preprocess)

trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

len(train_dataset)  # 96
# len(val_dataset)  # 28
# len(test_dataset)  # 12

132

In [7]:
from numpy.lib import shape_base
for batch in valloader:
    images, texts, labels = batch
    # Print the first batch
    print("Image Path:", images[0].shape)
    print("Text Description:", texts[0])
    print("Label: ", labels)
    break

Image Path: torch.Size([3, 224, 224])
Text Description: a photo of apple pie
Label:  tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])


# Training

In [20]:
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader)*EPOCH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [23]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import numpy as np

for epoch in range(EPOCH):
    print(f"Epoch: {epoch+1}")

    # Training loop
    model.train()
    train_total, train_correct, train_top1_correct, train_top3_correct = 0, 0, 0, 0
    pbar = tqdm(trainloader, total=len(trainloader))
    for batch in pbar:
        optimizer.zero_grad()

        images, texts, _ = batch
        texts = clip.tokenize(texts).to(device)
        images = images.to(device)

        logits_per_image, logits_per_text = model(images, texts)

        # Compute loss
        actual_batch_size = images.size(0)
        ground_truth = torch.arange(actual_batch_size).to(device)
        total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

        # Compute train accuracy
        train_correct += (logits_per_image.argmax(dim=1) == ground_truth).float().sum().item()
        train_total += images.size(0)

        total_loss.backward()

        if device == "cpu":
            optimizer.step()
        else :
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

        train_accuracy = 100 * train_correct / train_total
        pbar.set_description(f"Epoch {epoch+1}/{EPOCH}, Loss: {total_loss.item():.4f}, Train Acc: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    val_total, val_correct_top1, val_correct_top3 = 0, 0, 0

    with torch.no_grad():
        for batch in valloader:
            images, texts, labels = batch
            texts = clip.tokenize(texts).to(device)
            images = images.to(device)
            labels = labels.to(device)

            logits_per_image, _ = model(images, texts)
            # print("Logits per Image:", logits_per_image)

            # print("Predicted Labels:", logits_per_image.argmax(dim=1))
            # print("Actual Labels:", labels)

            # Compute top-1 and top-3 accuracy
            val_correct_top1 += (logits_per_image.argmax(dim=1) == labels).float().sum().item()
            _, top3_predictions = logits_per_image.topk(3, dim=1)
            val_correct_top3 += torch.any(top3_predictions == labels.view(-1, 1), dim=1).float().sum().item()

            val_total += images.size(0)

    # Compute and print validation accuracy
    val_accuracy_top1 = 100 * val_correct_top1 / val_total
    val_accuracy_top3 = 100 * val_correct_top3 / val_total

    print(f"Validation Top-1 Acc: {val_accuracy_top1:.2f}%, Validation Top-3 Acc: {val_accuracy_top3:.2f}%")



Epoch: 1


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Top-1 Acc: 3.03%, Validation Top-3 Acc: 9.09%
Epoch: 2


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Top-1 Acc: 3.03%, Validation Top-3 Acc: 9.09%
Epoch: 3


...

Validation Top-1 Acc: 3.03%, Validation Top-3 Acc: 9.09%
Epoch: 4


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Top-1 Acc: 3.03%, Validation Top-3 Acc: 9.09%
Epoch: 5


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Top-1 Acc: 3.03%, Validation Top-3 Acc: 9.09%


In [ ]:
torch.save(model.state_dict(), 'CLIP_4shot_v4.pth')
shutil.copy('CLIP_4shot_v4.pth', '/content/drive/MyDrive/Study/DL_CLIP/model/CLIP_4shot_v4.pth')

'/content/drive/MyDrive/Study/DL_CLIP/model/CLIP_4shot_v4.pth'

In [ ]:
loaded_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
loaded_model = loaded_model.to(torch.float32)
loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/Study/DL_CLIP/model/CLIP_v1_baseline.pth'))
loaded_model.to(device)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# 모델을 평가하기 전에 모델을 evaluation 모드로 설정
model.eval()

# 테스트 루프
test_correct = 0
test_total = 0
true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in testloader:
        images, texts, labels_tuple = batch
        texts = texts.squeeze(1)
        images = images.to(device)
        texts = texts.to(device)
        labels = labels_tuple[1].to(device)  # 레이블을 적절히 추출

        logits_per_image, _ = model(images, texts)

        # 예측값과 실제 레이블 저장
        _, predicted = logits_per_image.max(1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        # 정확도 계산
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

# 전체 테스트 세트에 대한 정확도 출력
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Precision, Recall, F1-score 계산
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Test Accuracy: 3.03%
Precision: 0.0061
Recall: 0.0303
F1-score: 0.0101


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
